In [1]:
from app.backend.core.file_processor import FileProcessor
from app.backend.core.analyzer import Analyzer

processor = FileProcessor()
analyzer = await Analyzer.create()


2025-07-14 21:23:36 | app.backend.services.groq | DEBUG | Retrieving Groq API key from Azure Key Vault...
2025-07-14 21:23:40 | app.backend.services.groq | DEBUG | Groq API key retrieved.


In [2]:
job = """
Job Summary
Do you have a passion for Artificial Intelligence (AI) and are looking to make a difference in how the latest clinical trials progress into the market?

 

Do you have a background in data science and/or computer science with experience working with AI tools for interactive AI applications across various IT systems? 

 

If so, we have an exciting opportunity for you at Medpace.

 

We are currently seeking a professional with experience with artificial intelligence (AI) tools such as NLP, LLM, IA etc. This professional will be programming and fine-tuning various AI tools and helping IT teams to implement them into new applications.

 

This professional will work collaboratively across the organization with multiple teams to improve the efficiency of company processes and provide insights and assistance to users.


Responsibilities

Design, implement, and deploy machine learning models and systems to solve complex problems and drive business outcomes;
Research, develop, and implement machine learning algorithms and models for tasks such as classification, regression, clustering, anomaly detection, and recommendation systems;
Research, develop, and implement AI tools such as NLP, LLM, and IA;
Collect, preprocess, and curate large datasets required for training generative models;
Experiment with different machine learning techniques and algorithms, including supervised, unsupervised, semi-supervised, reinforcement, and deep learning;
Design and optimize machine learning pipelines and workflows, incorporating techniques for data cleaning, feature engineering, model selection, and hyperparameter tuning; and
Develop scalable and efficient machine learning infrastructure and systems for training, testing, and deploying models in production environments.

Qualifications

Bachelor’s degree or higher in Artificial Intelligence, Computer or Data Science, or related field. Master’s degree preferred;
Proven experience implementing and fine-tuning Large Language Models or interactive AI applications (>1 year post-graduate experience);
Technical proficiency in programming languages and frameworks commonly used in NLP and AI (e.g., Python, TensorFlow, PyTorch);
Preferably several years of experience working with different AI capabilities and showcasing your passion both at work and outside work in the utilization of AI;
Excellent communication skills to collaborate effectively with cross-functional teams;
A passion for staying up-to-date with the latest advancements in NLP and AI technologies; and
Analytical thinker with great attention to detail. 

Medpace Overview

Medpace is a full-service clinical contract research organization (CRO). We provide Phase I-IV clinical development services to the biotechnology, pharmaceutical and medical device industries. Our mission is to accelerate the global development of safe and effective medical therapeutics through its scientific and disciplined approach. We leverage local regulatory and therapeutic expertise across all major areas including oncology, cardiology, metabolic disease, endocrinology, central nervous system, anti-viral and anti-infective. Headquartered in Cincinnati, Ohio, employing more than 5,000 people across 40+ countries.


Why Medpace?

People. Purpose. Passion. Make a Difference Tomorrow. Join Us Today.

 

The work we’ve done over the past 30+ years has positively impacted the lives of countless patients and families who face hundreds of diseases across all key therapeutic areas. The work we do today will improve the lives of people living with illness and disease in the future.

 

Cincinnati Perks

Cincinnati Campus Overview
Flexible work environment
Competitive PTO packages, starting at 20+ days
Competitive compensation and benefits package
Company-sponsored employee appreciation events
Employee health and wellness initiatives
Community involvement with local nonprofit organizations
Discounts on local sports games, fitness gyms and attractions
Modern, ecofriendly campus with an on-site fitness center
Structured career paths with opportunities for professional growth
Discounted tuition for UC online programs
Awards

Named a Top Workplace in 2024 by The Cincinnati Enquirer

Recognized by Forbes as one of America's Most Successful Midsize Companies in 2021, 2022, 2023 and 2024
Continually recognized with CRO Leadership Awards from Life Science Leader magazine based on expertise, quality, capabilities, reliability, and compatibility
"""
response = await analyzer.analyze_job(job)

2025-07-14 21:24:50 | app.backend.services.groq | INFO | Running structured query with function calling using ainvoke...
2025-07-14 21:24:54 | app.backend.services.groq | DEBUG | Beautified JSON args:
{
    "certificate": [],
    "domain": "Clinical trials",
    "educational_requirements": [{
        "fields": ["Artificial Intelligence", "Computer Science", "Data Science"],
        "level": "Bachelor's"
    }, {
        "fields": ["Artificial Intelligence", "Computer Science", "Data Science"],
        "level": "Master's"
    }],
    "employment_type": "Full-time",
    "experience": ["1+ year in AI tools such as NLP, LLM, IA"],
    "ideal_candidate_summary": "A professional with a passion for Artificial Intelligence and experience with AI tools, looking to make a difference in clinical trials.",
    "job_level": "Mid",
    "job_title": "AI Professional",
    "location_requirement": "Onsite in Cincinnati, OH",
    "responsibilities": ["Design, implement, and deploy machine learning model

In [ ]:
resume = await processor.extract_from_file("data/ExampleResume1.pdf")
response = await analyzer.analyze_candidate(resume)

2025-07-08 00:21:05 | app.backend.core.file_processor | INFO | Extracting text from local file: data/ExampleResume1.pdf
2025-07-08 00:21:05 | app.backend.core.file_processor | INFO | Using .PDF parser for file: data/ExampleResume1.pdf
2025-07-08 00:21:05 | app.backend.core.file_processor | INFO | Extracting text from 'data/ExampleResume1.pdf' using local PDF parser (pypdf)
2025-07-08 00:21:05 | app.backend.services.groq | INFO | Running structured query with function calling using ainvoke...
2025-07-08 00:21:08 | app.backend.services.groq | DEBUG | Beautified JSON args:
{
    "candidate_name": "Hedy Lamarr",
    "certificate": [],
    "comment": "Hedy Lamarr has a strong background in computer science and statistics, with experience in data engineering and software development. She has worked with various technologies and has a proven track record of delivering results in fast-paced environments.",
    "education": [{
        "field": "Computer Science, Statistics",
        "institutio

In [1]:
from app.backend.services.factory import AIServiceFactory

service = await AIServiceFactory.create_service()
response = await service.query("What is the area of the UK?")
print(response)

2025-07-07 00:38:58 | app.backend.services.groq | DEBUG | Retrieving Groq API key from Azure Key Vault...
2025-07-07 00:39:01 | app.backend.services.groq | DEBUG | Groq API key retrieved.


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000255A5E7B1C0>


2025-07-07 00:39:01 | app.backend.services.groq | INFO | Calling Groq model 'llama3-70b-8192' using ainvoke...
The total area of the United Kingdom (UK) is approximately 243,610 square kilometers (93,625 square miles). This includes:

* England: 130,279 km² (50,261 sq mi)
* Scotland: 78,772 km² (30,414 sq mi)
* Wales: 20,779 km² (8,023 sq mi)
* Northern Ireland: 14,160 km² (5,470 sq mi)

Note: These figures are approximate and may vary slightly depending on the source.


In [13]:
from datetime import date

prompt = f"""
You are an expert HR recruiter and resume screener. Your task is to extract relevant information from a resume and organize it into structured categories using the format below.

---

**Categories**:

1. **Synopsis** - A personal summary, description, or “about me” section.  
2. **Experience** - Includes jobs/internships under `<job>` and standalone projects under `<project>`.  
3. **Education** - Each degree appears under a `<degreeEntry>` with optional GPA and graduation year.  
4. **Skills** - Divided into technical and non-technical.

---

**Important Instructions**:
- If no synopsis is present, insert the word `Unknown` in the `<synopsis>` tag.
- Do **not** invent or infer missing information. Only extract what is explicitly present.
- Use the exact XML structure and tag names provided.
- All durations must use time units (e.g., `"3 months"`, `"2 years"`).
- Only include UTF-8 characters. Strip bullets, markdown, or decorative formatting.
    - If only a single month is present in the date range, the duration is `"1 month"`
    - For reference, `"Present"` in a job duration refers to {date.today()}

---

**Format your output as follows**:

```xml
<resume>
    <synopsis>
        INSERT SYNOPSIS INFORMATION HERE
    </synopsis>
    <experience>
        <job>
            <company>
                INSERT COMPANY NAME
            </company>
            <duration>
                INSERT JOB DURATION
            </duration>
            <description>
                INSERT JOB DESCRIPTION
            </description>
        </job>
        <!-- Repeat <job> as needed -->

        <project>
            <name>
                INSERT PROJECT NAME
            </name>
            <duration>
                INSERT PROJECT DURATION (if available)
            </duration>
            <description>
                INSERT PROJECT DESCRIPTION
            </description>
        </project>
        <!-- Repeat <project> as needed -->
    </experience>
    <education>
        <degreeEntry>
            <university>
                INSERT UNIVERSITY NAME
            </university>
            <degree>
                <level>
                    INSERT DEGREE LEVEL (e.g., Doctorate, Masters, Baccalaureate)
                </level>
                <name>
                    INSERT DEGREE NAME (e.g., Computer Science)
                </name>
            </degree>
            <graduationYear>
                INSERT GRADUATION YEAR (if available)
            </graduationYear>
            <gpa>
                INSERT GPA (if available)
            </gpa>
        </degreeEntry>
        <!-- Repeat <degreeEntry> as needed -->
    </education>
    <skills>
        <technical>
            <software>
                LIST SOFTWARE SKILLS
            </software>
            <languages>
                LIST PROGRAMMING LANGUAGES
            </languages>
            <frameworks>
                LIST FRAMEWORKS
            </frameworks>
            <tools>
                LIST TECHNICAL TOOLS
            </tools>
        </technical>
        <nontechnical>
            LIST NON-TECHNICAL SKILLS
        </nontechnical>
    </skills>
</resume>

---

You will now be given a resume in plain text. Extract and organize the information as described above.
**Resume**:  
{resume}
"""